In [34]:
import sys
import os

import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# add parent directory to path\n",
library_path = os.path.abspath('..')
if library_path not in sys.path:
    sys.path.append(library_path)

In [35]:
FOLDER = '/media/luis/LaCie/data1/PubMedPapers/'

df_semi = pd.read_csv(
    os.path.join(FOLDER, 'publications_filtered_v7_semicolons.csv')
)
df_semi['Affiliations']=df_semi['Affiliations'].astype(str)
df_semi['Country'] = ''

df_semi

,Affiliations,PubMed ID,Author,Country
0,"Mackenzie Wearables Research Hub, Charles Perk...",38852004,Le Wei,
1,"School of Health Sciences, Faculty of Medicin...",38852004,Le Wei,
2,"Mackenzie Wearables Research Hub, Charles Perk...",38852004,Matthew N Ahmadi,
3,"School of Health Sciences, Faculty of Medicin...",38852004,Matthew N Ahmadi,
4,School of Human Movement and Nutrition Science...,38852004,Stewart Trost,
...,...,...,...,...
42113,"Department of Biostatistics, Harvard School O...",23423446,Peter Kraft,
42114,Nuffield Department of Obstetrics and Gynaecol...,25328660,Stephen H Kennedy,
42115,"World Endometriosis Research Foundation, Lond...",25328660,Stephen H Kennedy,
42116,Nuffield Department of Obstetrics and Gynaecol...,25328660,Krina T Zondervan,


In [36]:
def country_encoder(affiliation: str, new_country: str, topos: list, old_country:str)-> str:
    
    for topo in topos:
        if topo in affiliation:
            return new_country

    return old_country

In [37]:
usa_locus = [
    "USA", "United States", "US", "Alabama", "California", "Albuquerque", "Ames IA", "Amherst MA", "Jackson MS",
    "Jacksonville FL", "Ann Arbor", "Albert Einstein College", "American Cancer Society", "Atlanta", "Massachusetts",
    "Boston", "Medford MA", "Milwaukee WI", "Arizona", "Baltimore", "Berkeley CA", "Bethesda MD", "Birmingham AL",
    "Boulder CO", "Aurora CO", "Cambridge MA", "Harvard MA", "Brookline MA", "Brownsville TX", "Bronx NY", "New Haven CT",
    "New Brunswick NJ", "Buffalo NY", "Burlington VT", "Califirnia", "New York", "Norwood MA", "Oakland", "Oregon",
    "Palo Alto CA", "San Francisco CA", "Chapel Hill NC", "Charleston SC", "Charlestown MA", "Charlotte NC", "Dallas",
    "Pennsylvania PA", "Pennsylvania", "Phoenix AZ", "Charlottesville VA", "Chicago", "Cincinnati", "Claremont CA",
    "Cleveland OH", "Collegeville PA", "Columbia SC", "Columbia University", "Providence RI", "Chevy Chase MD",
    "Puerto Rico", "Redwood City CA", "Triangle Park NC", "Columbus OH", "Corona CA", "Philadelphia", "Rochester MN",
    "Rockville MD", "Sacramento CA", "Salt Lake City", "San Antonio TX", "Seattle", "Rhode Island", "Brooklyn College",
    "Brigham and Womens Hospital", "Calico", "Sioux Center IA", "CedarsSinai", "Los Angeles", "Colorado", "Danville PA",
    "Davis CA", "Decatur GA", "Denver", "Stanford University", "Stanford Medical School", "Delaware", "West Virginia",
    "Louisville KY", "Brown University", "Blacksburg VA", "District of Columbia", "Downers Grove IL", "Duarte CA",
    "Duke University", "Tennessee", "Texas", "East Lansing", "Emory University", "Englewood CO", "Evanston IL",
    "George Washington University", "Icahn School", "Durham NC", "Farmington CT", "Florida", "Fort Collins CO",
    "Fort Worth TX", "Framingham", "Frederick MD", "Fremont CA", "Gainesville", "Gaithersburg MD", "Grand Rapids MI",
    "Glen Echo MD", "Hagerstown MD", "Hanover NH", "Harrison NJ", "Hartford CT", "Hawaii", "Mississippi", "Michigan",
    "Miami", "Utah", "Honolulu", "Houston", "Illinois", "Iowa", "Irvine CA", "Ithaca NY", "Johns Hopkins", "Kalamazoo MI",
    "Kansas", "Kentucky", "Prussia PA", "La Jolla", "Lancaster PA", "Las Vegas NV", "Lewisburg PA", "Lexington KY",
    "Lisle IL", "Pittsburgh PA", "Yale Universit", "Yale School of Medicine", "Wisconsin", "Winston Salem NC",
    "WinstonSalem NC", "Washington", "Waco TX", "Virginia", "Vermont", "Vanderbilt", "Maryland", "North Carolina",
    "Pittsburgh", "Tulane", "Tucson", "Torrance", "Wake Forest School of Medicine", "VA Palo Alto", "VA San Diego",
    "Tarrytown NY", "Long Beach CA", "Louisiana", "Madison WI", "Masschusetts", "Memphis TN", "Charles Bronfman Institute",
    "Harvard", "Lerner Research Institute", "Minneapolis", "Minnesota", "Missouri", "Moffitt Cancer Center", 
    "Morristown NJ", "Mountain View CA", "Nashville", "Marina del Rey", "Maywood IL", "South Carolina", "Tampa FL",
    "Monell Chemical Senses Center", "Blood Institute MA", "Taussig Cancer Institute", "Tallahassee", "Syracuse NY",
    "Redwood City", "Oklahoma", "Ohio", "Palo Alto", "Portland OR", "New Hyde Park NY", "New Hampshire", "New Orleans",
    "New Jersey", "North Haven CT", "Northeastern University", "UAB Lung Imaging Lab", "Stony Brook NY", "Stamford CT",
    "Stanford", "St Louis MO", "Connecticut", "ChanZuckerberg Biohub", "Cleveland", "Greenville NC", "San Diego CA",
    "Demoulas Center for Cardiac Arrhythmias", "Portland ME", "SUNY Downstate College", "SWOG Statistical Center",
    "Santa Cruz CA", "Scottsdale AZ", "Scripps", "Severn Health Solutions", "Severna Park MD", "Frank H. Netter MD School of Medicine"
]

In [38]:
esp_locus = ["Spain", "España",  "Barcelona", "Madrid", "Ramón y Cajal", "Asturias", "Pamplona", "Granada", "BCNAIM",
             "GRINCAVA", "Murcia", "Oviedo", "Bellvitge Biomedical Research Institute"]
fra_locus = ["France", "Marseille", "Bordeaux", "Paris", "Inserm", "Centre de Recherche des Cordeliers", "Créteil",
             "Dijon", "Lille", "Nantes", "Montpellier", "la Santé", "Lyon", "Villejuif", "CNRS", "Toulouse"]
aus_locus = ["Australia", "Melbourne", "Perth", "Cancer Council Victoria", "Curtin University", "Sydney", "Queensland",
             "Monash University", "Australi", "Royal Victorian Eye and Ear Hospital"]
den_locus = ["Denmark", "Aarhus", "Danish", "Danmark", "Faroe Islands", "Greenland", "Herlev and Gentofte Hospital",
             "Rigshospitalet", "Copenhagen"]
uk_locus  = ["United Kingdom", "UK", "Aberdeen", "Addenbrookes Hospital", "N Ireland", "Scotland", "Belfast", "Bristol",
            "Cardiff", "London", "NHS Trust", "NHS trust", "British", "Edinburgh", "Glasgow", "NHS Foundation Trust",
            "John Radcliffe Hospital", "Lancaster University", "Leeds", "Leicester", "Liverpool", "United ingdom",
            "United kingdom", "Manchester", "UNITED KINGDOM", "Wellcome Centre for Human Genetics", "Lothian",
            "Wellcome Trust Centre for Human Genetics", "Loughborough University", "Ormond Street Hospital", "NIHR",
            "MRC Integrative Epidemiology", "MRC Population Health Research", "MRCBHF", "MRCPHE", "University of Exeter",
            "Medical Research Council", "BHF Data Science Centre", "BRC Oxford Haematology", "University of Southampton",
            "Francis Crick Institute", "Robertson Center for Biostatistics", "Midlothian", "Worcestershire",
            "UCL Institute of Child Health", "Swansea University", "Gibraltar", "Nuffield", "University of Cambridge",
            "University of Oxford"]
civ_locus = ["Côte dIvoire"]
uae_locus = ["UAE", "United Arab Emirates", "Abu Dhabi"]
nig_locus = ["Nigeria", "Abuja", "Lagos"]
chn_locus = ["China", "Beijing", "CHINA", "Chinese", "Central South University", "Changping", "Chengdu", "Dongfang",
             "Xian Jiaotong", "Tongxiang", "Fudan University", "Guangzhou", "Guangdong", "Harbin", "Heilongjiang",
             "Furong", "Guangxi", "Hong Kong", "Hongqiao", "Dongguan", "Hunan", "Jiangsu", "Jinan", "Zhengzhou",
             "Zhongshan", "Liaoning", "Liuzhou", "Zhejiang", "Yichang", "Xiangya", "Wuhan", "Tianjin", "Tsinghua",
             "Wannan Medical College", "Nanchang", "Shandong", "Li Ka Shing Faculty of Medicine", "Xuanwu", "Tongji",
             "Soochow University", "Oujiang", "Peking", "Pengzhou", "Yueyang", "Yinchuan", "Yantai", "Fujian",
             "Sun Yatsen University", "Shiyan", "Shidong", "Shenzhen", "Shengjing", "Guangan", "Huanggang", "Hubei",
             "Huashan", "Nanjing", "Shaanxi", "Shanghai", "Southern Medical University", " State Key Laboratory of Organ Failure Research",
             " National Clinical Research Center for Kidney Disease"]
eth_locus = ["Ethiopia"]
jpn_locus = ["Japan", "Tokyo", "Osaka", "Chiba University", "Tokushima", "Tohoku", "Fukuoka", "Hiroshima",
             "Iwate Medical University", "Kanagawa", "Nagoya", "Gifu", "Otsu", "RIKEN Center", "Saga University"]
swe_locus = ["Sweden", "Stockholm", "Malmö", "Lund", "Umeå", "Karolinska Institute", "Linköping", "Uppsala", "Sweded",
             "Karolinska"]
nth_locus = ["Netherlands", "Amsterdam", "EMGO", "Vrije Universiteit", "Leiden", "Maastricht", "Utrecht", "Hubrecht",
             "GRIAC", "Julius Center for Health Sciences", "VU University Medical Center", "Rotterdam", "Nederland",
             "Bilthoven", "Radboud"]
ger_locus = ["Germany", "Heidelberg", "Augsburg", "Berlin", "Munich", "Deutschland", "Essen", "Freiburg", "Fraunhofer",
             "German", "Greifswald", "Helmholtz", "Erlangen", "Hamburg", "DKFZ", "DZHK", "LIFE Child", "MaxPlanck",
             "Neuherberg", "CharitéUniversität", "FriedrichAlexanderUniversität", "HumboldtUniversität", 
             "LudwigMaximilian"]
aut_locus = ["Austria", "Graz", "Innsbruck"]
tur_locus = ["Turkey"]
gre_locus = ["Greece", "University of Crete"]
ita_locus = ["Italy", "Ragusa","Sassari", "Rome", "Bergamo", "Roma", "DIBRIS", "Torino", "Florence", "Fondazione IRCCS",
             "ITALY", "Italia", "Itlay", "Milan", "Padua", "Pavia", "Università Cattolica Sacro Cuore", "Udine"]
col_locus = ["Colombia", "Bogotá", "Bogota"]
rus_locus = ["Russia", "Moscow"]
can_locus = ["Canada", "British Columbia", "Ontario", "Québec", "Quebec", "Toronto", "Vancouver", "Lady Davis Institute",
             "Montreal", "Hamilton Health Sciences", "McGill University", "Sunnybrook Research Institute"]
nor_locus = ["Norway", "Oslo", "Bergen", "Norwegian"]
bel_locus = ["Belgium", "Leuven"]
nz_locus  = ["New Zealand", "Auckland"]
fin_locus = ["Finland", "Oulu", "FINLAND", "Finlan", "Helsinki", "Kuopio", "Finnish", "Pirkanmaa"]
isr_locus = ["Israel"]
arg_locus = ["Argentina", "Buenos Aires"]
per_locus = ["Perú", "Peru"]
skr_locus = ["South Korea", "Republic of Korea", "Busan", "Seoul", "Cheongjusi", "Chungnam", "Daegu", "Daejeon", "Goyangsi",
             "Gwangju", "Gyeonggido", "Ulsan", "Hwasun", "Korea University", "Uijeongbu Eulji", "Yonsei", "South of Korea",
             "Seongnam"]
bra_locus = ["Brazil", "Brasil"]
chi_locus = ["Chile", "Talca"]
ind_locus = ["India"]
hun_locus = ["Hungary", "Budapest"]
tai_locus = ["Taiwan", "Taipei", "Chung Shan Medical", "Taichung", "Hualien", "Kaohsiung", "Taoyuan", "Tamkang", "Hsinchu",
             "National YangMing University"]
ksa_locus = ["Saudi Arabia", "Riyadh"]
por_locus = ["Portugal", "Aveiro", "Coimbra"]
swi_locus = ["Switzerland", "Botnar Research Centre", "Zurich", "Bern", "Geneva", "Swiss", "Lausanne"]
egy_locus = ["Egypt"]
bru_locus = ["Brunei"]
mal_locus = ["Mali"]
tha_locus = ["Thailand"]
bar_locus = ["Barbados"]
cyp_locus = ["Cyprus"]
sin_locus = ["Singapore", "Nanyang"]
ina_locus = ["Indonesia"]
bos_locus = ["Bosnia and Herzegovina"]
bah_locus = ["Bahrain"]
cam_locus = ["Cameroon", "Cameroun"]
rsa_locus = ["South Africa", "University of the Witwatersrand"]
phi_locus = ["Philippines"]
ban_locus = ["Bangladesh", "Dhaka"]
qat_locus = ["Qatar", "Doha"]
ire_locus = ["Ireland", "Dublin"]
mex_locus = ["Mexico", "México"]
est_locus = ["Estonia", "Tartu"]
slk_locus = ["Slovak"]
cz_locus  = ["Czech", "Prague"]
gha_locus = ["Ghana"]
ice_locus = ["Iceland", "Kopavogur"]
slv_locus = ["Slovenia"]
irn_locus = ["Iran"]
pak_locus = ["Pakistan"]
jam_locus = ["Jamaica"]
pol_locus = ["Poland"]
cro_locus = ["Croatia"]
gam_locus = ["Gambia"]
uga_locus = ["Uganda"]
ukr_locus = ["Ukraine"]
mly_locus = ["Malaysia"]
mlw_locus = ["Malawi"]
srb_locus = ["Serbia"]
mzb_locus = ["Mozambique"]
mac_locus = ["Macedonia"]
dom_locus = ["Dominican Republic", "Dominicana"]
blr_locus = ["Belarus"]
lao_locus = ["Laos"]
lth_locus = ["Lithuania"]
uzb_locus = ["Uzbekistan"]
kuw_locus = ["Kuwait"]
rom_locus = ["Romania", "Ana Aslan International Foundation"]

In [39]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='USA', 
        topos=usa_locus, 
        old_country=x['Country']
    ), axis=1
)

In [40]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='China', 
        topos=chn_locus, 
        old_country=x['Country']
    ), axis=1
)

In [41]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='UK', 
        topos=uk_locus, 
        old_country=x['Country']
    ), axis=1
)

In [42]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Australia', 
        topos=aus_locus, 
        old_country=x['Country']
    ), axis=1
)

In [43]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Canada', 
        topos=can_locus, 
        old_country=x['Country']
    ), axis=1
)

In [44]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Germany', 
        topos=ger_locus, 
        old_country=x['Country']
    ), axis=1
)

In [45]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Norway', 
        topos=nor_locus, 
        old_country=x['Country']
    ), axis=1
)

In [46]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Denmark', 
        topos=den_locus, 
        old_country=x['Country']
    ), axis=1
)

In [47]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Spain', 
        topos=esp_locus, 
        old_country=x['Country']
    ), axis=1
)

In [48]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='France', 
        topos=fra_locus, 
        old_country=x['Country']
    ), axis=1
)

In [49]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Hungary', 
        topos=hun_locus, 
        old_country=x['Country']
    ), axis=1
)

In [50]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Finland', 
        topos=fin_locus, 
        old_country=x['Country']
    ), axis=1
)

In [51]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Taiwan', 
        topos=tai_locus, 
        old_country=x['Country']
    ), axis=1
)

In [52]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Belgium', 
        topos=bel_locus, 
        old_country=x['Country']
    ), axis=1
)

In [53]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Netherlands', 
        topos=nth_locus, 
        old_country=x['Country']
    ), axis=1
)

In [54]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Sweden', 
        topos=swe_locus, 
        old_country=x['Country']
    ), axis=1
)

In [55]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Switzerland', 
        topos=swi_locus, 
        old_country=x['Country']
    ), axis=1
)

In [56]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Italy', 
        topos=ita_locus, 
        old_country=x['Country']
    ), axis=1
)

In [57]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Singapore', 
        topos=sin_locus, 
        old_country=x['Country']
    ), axis=1
)

In [58]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Japan', 
        topos=jpn_locus, 
        old_country=x['Country']
    ), axis=1
)

In [59]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Portugal', 
        topos=por_locus, 
        old_country=x['Country']
    ), axis=1
)

In [60]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Estonia', 
        topos=est_locus, 
        old_country=x['Country']
    ), axis=1
)

In [61]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Russia', 
        topos=rus_locus, 
        old_country=x['Country']
    ), axis=1
)

In [62]:
df_semi['Country'] = df_semi.apply(
    lambda x: country_encoder(
        affiliation=x['Affiliations'], 
        new_country='Poland', 
        topos=pol_locus, 
        old_country=x['Country']
    ), axis=1
)

In [63]:
empty = df_semi[df_semi['Country'] == ''].reset_index(drop=True)
empty

,Affiliations,PubMed ID,Author,Country
0,The Second People&apos,38787375,Xiaoxiao Han,
1,The Second People&apos,38787375,Shuo Yu,
2,and the,38737007,Jagat Narula,
3,"Department of Internal Medicine, Seoul Nationa...",38697272,Eue-Keun Choi,
4,"Department of Internal Medicine, Seoul Nation...",38697272,Eue-Keun Choi,
...,...,...,...,...
6283,CIBER Epidemiology and Public Health CIBERESP,23857962,E Molina Montes,
6284,WHO Collaborating Center for Food and Nutritio...,23857962,A Trichopoulou,
6285,"Hellenic Health Foundation, Athens, Greece",23857962,A Trichopoulou,
6286,"Hellenic Health Foundation, Athens, Greece",23857962,D Trichopoulos,
